In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from matplotlib.patches import Patch
from sklearn.decomposition import PCA
from tensorflow import keras
import pandas as pd
import random

In [4]:
def set_seed(seed=42):
  random.seed(seed)
  np.random.seed(seed)
  tf.experimental.numpy.random.seed(seed)

set_seed()
project_dir = os.getcwd()


In [3]:
model_mn2 = keras.applications.MobileNetV2(
    input_shape=None, alpha=1.0, include_top=True, weights='imagenet',
    input_tensor=None, pooling=None, classes=1000
)
layer_name = 'block_15_add'
feature_extractor = keras.models.Model(inputs=model_mn2.input, outputs=model_mn2.get_layer(layer_name).output)

In [4]:
models_path = os.path.join(project_dir, 'models')
model_mobilenetv2_regularized = tf.keras.models.load_model(os.path.join(models_path, 'model_mobilenetv2_regularized_plantdoc'))
model_squeeze_excite_resnet = tf.keras.models.load_model(os.path.join(models_path, 'model_squeeze_excite_resnet_plantdoc'))
model_pca_based = tf.keras.models.load_model(os.path.join(models_path, 'model_pca_based_plantdoc'))
model_pca_based_sepconv = tf.keras.models.load_model(os.path.join(models_path, 'model_pca_based_sepconv_plantdoc'))
model_lda_based = tf.keras.models.load_model(os.path.join(models_path, 'model_lda_based_plantdoc'))
model_lda_based_sepconv = tf.keras.models.load_model(os.path.join(models_path, 'model_lda_based_sepconv_plantdoc'))

In [29]:
diseases_names = []
dataset_path = os.path.join(project_dir, 'datasets', 'encoded', 'PlantVillage')

for diseases in os.listdir(os.path.join(dataset_path, 'train')):
    diseases_names += [diseases.split('.npy')[0]]

diseases_names.sort()
plant_name_to_id = {plant: i for i, plant in enumerate(diseases_names)}

In [30]:
instances = {}
labels = {}

for set_kind in ["test", "train", "val"]:
    X_per_class = []
    y_per_class = []

    for i, file in enumerate(os.listdir(os.path.join(dataset_path, set_kind))):
        print(f"\r[{i}] {set_kind}/{file}", end='')
        plants_encoded = np.load(os.path.join(dataset_path, set_kind, file))
        plant_name = file.split('.npy')[0]

        X_per_class += [plants_encoded.reshape([plants_encoded.shape[0], 49, 160])]
        y_per_class += [plant_name_to_id[plant_name] for _ in range(plants_encoded.shape[0])]

    instances[set_kind] = np.concatenate(X_per_class)
    labels[set_kind] = np.array(y_per_class)

[26] val/Tomato Septoria leaf spot.npyyyy

In [31]:
n_classes = len(diseases_names)

X_train = instances['train']
y_train = np.eye(n_classes)[labels['train']]


X_valid = instances['val']
y_valid = np.eye(n_classes)[labels['val']]

X_test = instances['test']
y_test = np.eye(n_classes)[labels['test']]

In [8]:
class PCATransformer(keras.layers.Layer):
    def __init__(self, means, components, **kwargs):
        super().__init__(**kwargs)
        self.all_means = tf.reshape(tf.transpose(means), [1, 49, 160])
        self.all_components = tf.transpose(components, perm=[0, 2, 1])

    @tf.function
    def call(self, X):
        return tf.transpose(tf.map_fn(
            lambda inp: inp[0] @ inp[1],
            (tf.transpose(X - self.all_means, perm=[2, 0, 1]), self.all_components),
            fn_output_signature=tf.TensorSpec(shape=[None, 5])
        ), perm=[1, 2, 0])

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape((batch_input_shape[0], 5, 160))

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "mean": self.all_means.numpy(), "components": self.all_components.numpy()}



In [9]:
pca_per_filter = [
    PCA(n_components=5).fit(X_train[..., i])
    for i in range(X_train.shape[-1])
]

all_means = np.array([filter.mean_ for filter in pca_per_filter])
all_components = np.array([filter.components_ for filter in pca_per_filter])
pca_transformation = PCATransformer(tf.constant(all_means), tf.constant(all_components))



In [30]:
model_lda_based_sepconv.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_2 (Batc  (None, 5, 160)           640       
 hNormalization)                                                 
                                                                 
 separable_conv1d (Separable  (None, 1, 320)           156320    
 Conv1D)                                                         
                                                                 
 elu_2 (ELU)                 (None, 1, 320)            0         
                                                                 
 flatten (Flatten)           (None, 320)               0         
                                                                 
 dropout_2 (Dropout)         (None, 320)               0         
                                                                 
 batch_normalization_3 (Batc  (None, 320)             

In [13]:
count = 0
for i in os.listdir('./datasets/original/PlantDoc/test'):
    count += len(os.listdir('./datasets/original/PlantDoc/train/' + i))

print(count)

2334
